In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
from networkx import shortest_path_length
import json
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

class ProductRecommendationSystem:
    def __init__(self):
        self.graph = nx.DiGraph()
        self.description_model = SentenceTransformer("dragonkue/bge-m3-ko")
        
    def create_category_graph(self, data_path):
        """카테고리 계층 구조 그래프를 생성합니다."""
        data = pd.read_csv(data_path)
        categories = data[['카테고리1', '카테고리2', '카테고리3', '카테고리4']]
        categories = categories.replace('None', pd.NA)
        category_paths = categories.apply(lambda x: '/'.join(x.dropna().astype(str)), axis=1)
        category_paths = category_paths.apply(lambda x: x.split('/'))
        
        for path in category_paths:
            for i in range(len(path) - 1):
                self.graph.add_edge(path[i], path[i + 1])
        
        if 'Root' not in self.graph:
            self.graph.add_node('Root')
            for node in self.graph.nodes():
                if node != 'Root' and not list(self.graph.predecessors(node)):
                    self.graph.add_edge('Root', node)

    def visualize_category_hierarchy(self, output_path="category_hierarchy.png"):
        """카테고리 계층 구조를 시각화합니다."""
        plt.figure(figsize=(15, 10))
        pos = nx.spring_layout(self.graph)
        nx.draw(self.graph, pos, with_labels=True, node_size=1500, 
                node_color="skyblue", font_size=10, font_weight="bold")
        plt.savefig(output_path)
        plt.close()
                    
    def wu_palmer_similarity(self, category_path1, category_path2):
        """카테고리 계층 구조에서 Wu-Palmer 유사도를 계산합니다."""
        try:
            if category_path1 == category_path2:
                return 1.0

            common_depth = 0
            for i in range(min(len(category_path1), len(category_path2))):
                if category_path1[i] == category_path2[i]:
                    common_depth = i + 1
                else:
                    break

            if common_depth == 0:
                return 0

            depth1 = len(category_path1)
            depth2 = len(category_path2)

            similarity = (2 * common_depth) / (depth1 + depth2)
            return similarity

        except nx.NodeNotFound:
            return 0

    def compute_description_similarity(self, target_embedding, candidate_embedding):
        """사전 계산된 임베딩을 활용한 유사도 계산"""
        return cosine_similarity([target_embedding], [candidate_embedding])[0, 0]
    
    def create_user_purchase_history(self, orders_paths, item_profiles):
        """여러 주문 파일로부터 사용자별 구매 이력을 생성합니다."""
        user_purchase_history = defaultdict(list)
        user_item_history = defaultdict(set)

        for orders_path in orders_paths:
            df = pd.read_csv(orders_path)

            for _, row in df.iterrows():
                customer_id = row['고객식별ID']
                item_seq = str(row['상품ID'])

                if item_seq in item_profiles:
                    categories = item_profiles[item_seq]['categories']
                    if categories and categories[0]:
                        user_purchase_history[customer_id].append(categories)  # 카테고리만 저장
                        user_item_history[customer_id].add(item_seq)

        return dict(user_purchase_history), dict(user_item_history)
    
    def user_similarity(self, user_history1, user_history2):
        """두 사용자 구매 이력 간 유사도 계산 - 카테고리 유사도만 사용"""
        similarity_sum = 0
        comparisons = 0
        
        for cat1 in user_history1:
            for cat2 in user_history2:
                # 카테고리 유사도만 계산
                similarity = self.wu_palmer_similarity(cat1, cat2)
                similarity_sum += similarity
                comparisons += 1
                
        return similarity_sum / comparisons if comparisons else 0

    
    def recommend_collaborative(self, purchase_history, item_history, item_profiles, target_user, top_n=10, top_k_users=10, number_filter=None):
        """협업 필터링 기반 추천 - 카테고리 우선, 설명 유사도는 보조적으로 사용"""
        user_similarities = {}
        
        # 카테고리 유사도만으로 유사 사용자 찾기
        for other_user, history in purchase_history.items():
            if other_user != target_user:
                similarity = self.user_similarity(purchase_history[target_user], history)
                user_similarities[other_user] = similarity
        
        # 유사도가 높은 상위 k명의 사용자 선택
        top_similar_users = sorted(user_similarities.items(), key=lambda x: x[1], reverse=True)[:top_k_users]
        
        recommendations = []
        purchased_item_names = {item_profiles[item_id]['name'].rsplit(' ', 1)[0] for item_id in item_history[target_user]}
        recommended_item_ids = set()  # 이미 추천된 상품 ID를 저장할 집합

        # 유사 사용자들이 구매한 상품 평가
        for similar_user, user_similarity in top_similar_users:
            for item_id in item_history[similar_user]:
                # 이미 구매한 상품 제외 및 동일 이름의 다른 버전 상품 제외
                item_name = item_profiles[item_id]['name']
                base_item_name = item_name.rsplit(' ', 1)[0]  # 숫자 부분 제외한 이름
                
                if base_item_name in purchased_item_names:
                    continue  # 같은 이름의 다른 버전 제외
                
                if item_id not in item_history[target_user]:
                    # 이미 추천된 상품인지 확인
                    if item_id in recommended_item_ids:
                        continue  # 이미 추천된 상품이면 제외
                    
                    # 숫자 필터링 적용
                    if number_filter and not item_name.endswith(number_filter):
                        continue  # 지정된 숫자로 끝나지 않으면 제외
                    
                    item_data = item_profiles[item_id]
                    item_categories = item_data['categories']
                    
                    # 카테고리 유사도 계산
                    max_category_similarity = 0
                    max_description_similarity = 0
                    
                    for purchased_categories in purchase_history[target_user]:
                        if all(node in self.graph for node in item_categories) and \
                        all(node in self.graph for node in purchased_categories):
                            category_similarity = self.wu_palmer_similarity(item_categories, purchased_categories)
                            max_category_similarity = max(max_category_similarity, category_similarity)
                    
                    # 설명 유사도 계산 (임베딩이 있는 경우에만)
                    if 'embedding' in item_data:
                        for purchased_item_id in item_history[target_user]:
                            if 'embedding' in item_profiles[purchased_item_id]:
                                description_similarity = self.compute_description_similarity(
                                    item_data['embedding'],
                                    item_profiles[purchased_item_id]['embedding']
                                )
                                max_description_similarity = max(max_description_similarity, description_similarity)
                    
                    if max_category_similarity > 0:
                        recommendations.append({
                            'item_id': item_id,
                            'name': item_data['name'],
                            'category_similarity': max_category_similarity,
                            'description_similarity': max_description_similarity
                        })
                        recommended_item_ids.add(item_id)  # 추천된 상품 ID를 집합에 추가
        
        # 카테고리 유사도를 첫 번째 기준으로, 설명 유사도를 두 번째 기준으로 정렬
        recommendations.sort(key=lambda x: (x['category_similarity'], x['description_similarity']), reverse=True)
        return recommendations[:top_n]




def get_category_path_string(categories):
    """카테고리 리스트를 문자열로 변환합니다."""
    return ' > '.join(categories) if categories else ''

def load_item_profiles(json_path):
    """JSON 파일에서 item_profiles를 불러옵니다."""
    with open(json_path, 'r', encoding='utf-8') as f:
        item_profiles = json.load(f)
    return item_profiles

def main():
    json_path = 'item_profiles_with_embeddings.json'
    item_profiles = load_item_profiles(json_path)
    
    recommender = ProductRecommendationSystem()
    recommender.create_category_graph("left_joined_file.csv")
    
    orders_paths = ['order1_df.csv', 'order2_df.csv', 'order3_df.csv', 'order4_df.csv', 'order5_df.csv']
    purchase_history, item_history = recommender.create_user_purchase_history(orders_paths, item_profiles)
    
    # 사용자 입력 추가
    user_id = input("추천을 받을 사용자 ID를 입력하세요: ")
    number_filter = input("필터링할 숫자를 입력하세요 (1-5 중 하나): ")

    if user_id in purchase_history:
        print(f"\n{'='*80}")
        print(f"사용자 {user_id}의 추천 상품 (필터링 기준: 이름이 '{number_filter}'로 끝나는 상품):")
        recommendations = recommender.recommend_collaborative(
            purchase_history, item_history, item_profiles, user_id, number_filter=number_filter
        )
        
        for rec in recommendations:
            print(f"상품 이름: {rec['name']}")
            print(f"카테고리: {get_category_path_string(item_profiles[rec['item_id']]['categories'])}")
            print(f"카테고리 유사도: {rec['category_similarity']:.4f}")
            print(f"설명 유사도: {rec['description_similarity']:.4f}")
            print('-' * 40)
    else:
        print("입력한 사용자 ID에 대한 구매 이력이 없습니다.")


if __name__ == "__main__":
    main()